# Import libraries


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.ion()


import mne
mne.set_log_level('error')


import os
import random
import shutil

SEED = 42
np.random.seed(SEED)
random.seed(SEED)

pd.options.display.max_columns = None # показываем все колонки

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import xgboost as xgb

In [ ]:
def load_epochs_to_numpy(file_path):
    shutil.unpack_archive(file_path, extract_dir=os.path.dirname(file_path), format="zip")
    fif_path = os.path.splitext(file_path)[0] + ".fif"
    epochs = mne.read_epochs(fif_path, preload=True)
    os.remove(fif_path)

    # --- Визуализация событий ---
    fig, ax = plt.subplots(figsize=[15, 5])
    mne.viz.plot_events(
        epochs.events,
        epochs.info["sfreq"],
        event_id=epochs.event_id,
        axes=ax
    )
    plt.show()

    # --- Формирование X, y ---
    X = epochs.get_data(copy=False)        # (epochs, channels, time)
    y = epochs.events[:, 2]                # метки классов

    shape = X.shape
    n_epochs, n_channels, n_times = shape
    X = X.reshape(n_epochs, n_channels * n_times)

    print(f"X исходная: {shape} (эпохи, каналы, время)")
    print(f"X после reshape: {X.shape} (эпохи, признаки)")
    print(f"y: {y.shape} (метки классов)")

    return X, y

In [ ]:
xgb_classifier = xgb.XGBClassifier(
    objective='binary:logistic',
    n_estimators=100,
    learning_rate=0.2,
    max_depth=7,
    eval_metric='logloss',
    random_state=42
)

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', xgb_classifier)
])

In [ ]:
def run_transfer(X_train, y_train, X_test, y_test, label):
    # пересоздаем пайплайн при каждом запуске
    xgb_classifier = xgb.XGBClassifier(
        objective='binary:logistic',
        n_estimators=100,
        learning_rate=0.2,
        max_depth=7,
        eval_metric='logloss',
        random_state=42
    )

    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('clf', xgb_classifier)
    ])

    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)

    metrics = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred, pos_label=1),
        'recall': recall_score(y_test, y_pred, pos_label=1),
        'f1': f1_score(y_test, y_pred, pos_label=1)
    }

    return pd.DataFrame([metrics], index=[label]).round(2)

# Load files

In [ ]:
file_path = r'epochs\sam40_full_epochs.zip'
sam40_X, sam40_y = load_epochs_to_numpy(file_path)

In [ ]:
file_path = r'epochs\distinguishing_epochs.zip'
distinguishing_X, distinguishing_y = load_epochs_to_numpy(file_path)

file_path = r'epochs\sam40_stroop_epochs.zip'
stroop_X, stroop_y = load_epochs_to_numpy(file_path)

In [ ]:
file_path = r'epochs\sam40_arithmetic_epochs.zip'
arithmetic_X, arithmetic_y = load_epochs_to_numpy(file_path)

In [ ]:
file_path = r'epochs\sam40_mirror_epochs.zip'
mirror_X, mirror_y = load_epochs_to_numpy(file_path)

# Table

In [ ]:
# --- Cross-task внутри SAM40 --- #
res1 = run_transfer(stroop_X, stroop_y,
                    arithmetic_X, arithmetic_y,
                    "Stroop → Arithmetic")

res2 = run_transfer(stroop_X, stroop_y,
                    mirror_X, mirror_y,
                    "Stroop → Mirror")

res3 = run_transfer(arithmetic_X, arithmetic_y,
                    stroop_X, stroop_y,
                    "Arithmetic → Stroop")

res4 = run_transfer(arithmetic_X, arithmetic_y,
                    mirror_X, mirror_y,
                    "Arithmetic → Mirror")

res5 = run_transfer(mirror_X, mirror_y,
                    stroop_X, stroop_y,
                    "Mirror → Stroop")

res6 = run_transfer(mirror_X, mirror_y,
                    arithmetic_X, arithmetic_y,
                    "Mirror → Arithmetic")


# --- Cross-dataset: SAM40 ↔ Distinguishing --- #
res7 = run_transfer(distinguishing_X, distinguishing_y,
                    sam40_X, sam40_y,
                    "Distinguishing → SAM40")

res8 = run_transfer(sam40_X, sam40_y,
                    distinguishing_X, distinguishing_y,
                    "SAM40 → Distinguishing")


# --- Cross-dataset: Distinguishing → SAM40 subtasks --- #
res9 = run_transfer(distinguishing_X, distinguishing_y,
                    stroop_X, stroop_y,
                    "Distinguishing → Stroop")

res10 = run_transfer(distinguishing_X, distinguishing_y,
                     arithmetic_X, arithmetic_y,
                     "Distinguishing → Arithmetic")

res11 = run_transfer(distinguishing_X, distinguishing_y,
                     mirror_X, mirror_y,
                     "Distinguishing → Mirror")


# --- Cross-dataset: SAM40 subtasks → Distinguishing --- #
res12 = run_transfer(stroop_X, stroop_y,
                     distinguishing_X, distinguishing_y,
                     "Stroop → Distinguishing")

res13 = run_transfer(arithmetic_X, arithmetic_y,
                     distinguishing_X, distinguishing_y,
                     "Arithmetic → Distinguishing")

res14 = run_transfer(mirror_X, mirror_y,
                     distinguishing_X, distinguishing_y,
                     "Mirror → Distinguishing")


# --- Итоговая таблица --- #
metrics_table = pd.concat([
    res1, res2, res3, res4, res5, res6,   # cross-task
    res7, res8,                           # full dataset → full dataset
    res9, res10, res11,                   # Distinguishing → SAM40 subtasks
    res12, res13, res14                   # SAM40 subtasks → Distinguishing
])

metrics_table